In [1]:
import numpy as np
import pandas as pd
from botorch.test_functions.synthetic import Ackley, AckleyMixed, Labs, Griewank
import torch
from sklearn.decomposition import PCA
from tqdm import tqdm

import sys
sys.path.append('../')
from roelfes_emulator import RoelfesEmulator
from vae import Autoencoder, WeightedAutoencoder
from bayesian_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import gc
from joblib import Parallel, delayed

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate_random_inputs(bounds, q, discrete_inds=None):
    lower, upper = bounds  
    generated_samples = lower + (upper - lower) * torch.rand((q, bounds.shape[1]))
    if discrete_inds is not None:
        generated_samples[:, discrete_inds] = torch.round(generated_samples[:, discrete_inds])
    return generated_samples

In [3]:
def bayes_opt_roelfes_coef(objs, q, q_sampling_method="Monte Carlo"):
    lin_y_names= ['y1','y2']
    dec_exp_y_names = ['y3','y4','y5','y6']
    all_names = lin_y_names + dec_exp_y_names

    lin_df = objs.drop(dec_exp_y_names,axis=1)
    dec_exp_df = objs.drop(lin_y_names, axis=1)

    q1 = q // 2
    q2 = q - q1
    if q == 1:
        q1 = 1

    lin_bo_model = BayesianOptimization().fit(lin_df, y=lin_y_names)
    c1 = lin_bo_model.candidates(q1, export_df=True, q_sampling_method=q_sampling_method)
    dec_exp_bo_model = BayesianOptimization().fit(dec_exp_df, y=dec_exp_y_names, 
                                                optim_direc=['max','max', 'min','max'])
    c2 = dec_exp_bo_model.candidates(q2, export_df=True,q_sampling_method=q_sampling_method)
    cand = pd.concat((c1, c2)).fillna(0)
    pred = cand[all_names].to_numpy()
    cand = cand.drop(all_names, axis=1).to_numpy()
    if q == 1:
        choice = np.random.randint(0,2)
        return cand[choice,:], pred[choice,:]
        
    return cand, pred

In [4]:
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

np.random.seed(42)

starting_samples_n = 96
bo_iter_amount = 25
dim_red_list = [PCA, None, Autoencoder, WeightedAutoencoder]
test_component_choice_list = [2, 16, 32]
sampling_strategies = ["Monte Carlo", "Sequential"]
cv_amount = 5
q_list = [1, 2, 4, 16]

# Define your functions with names
function_list = [
    ("RoelfesCoef", RoelfesEmulator(dim=50, objective="coef", space_complexity=100, use_torch=True)),
    ("Ackley", Ackley(dim=50)),
    ("AckleyMixed", AckleyMixed(dim=53)),
    ("Labs", Labs(dim=50)),
    ("Griewank", Griewank(dim=50)),
    ("RoelfesMaxYield", RoelfesEmulator(dim=50, objective="max yield", space_complexity=100, use_torch=True)),
    ("RoelfesNormGrad", RoelfesEmulator(dim=50, objective="normalized yield gradient", space_complexity=100, use_torch=True)),
]

# Define optimization directions
optim_direc_map = {
    "RoelfesCoef": ["max", "max", "max", "max", "min", "max"],
    "Ackley": ["min"],
    "AckleyMixed": ["min"],
    "Labs": ["min"],
    "Griewank": ["min"],
    "RoelfesMaxYield": ["max"],
    "RoelfesNormGrad": ["max"],
}

# Results storage
results = {}
data = {}

In [5]:
def run_single_experiment(function_name, function, optim_direc, weights,
                          cv_iter, x_np_backup, y_np_backup,
                          dr_method, dr_name, n_components,
                          q, q_method):
    from sklearn.metrics import mean_absolute_error

    x_np, y_np = x_np_backup.copy(), y_np_backup.copy()
    q_key = f"q{q}_{q_method or 'None'}"

    best_vals = []
    mae_pred_vals = []
    recon_mae_vals = []

    for bo_iter in range(bo_iter_amount):
        # Apply dimensionality reduction if specified
        if dr_method:
            dr_model = dr_method(n_components=n_components)
            if dr_name == "WeightedAutoencoder":
                x_np_dr = dr_model.fit_transform(x_np, y_np, optim_direc=optim_direc)
            else:
                x_np_dr = dr_model.fit_transform(x_np)
            x_np_recon = dr_model.inverse_transform(x_np_dr)
            recon_mae_vals.append(mean_absolute_error(x_np_recon, x_np))
        else:
            x_np_dr = x_np
            recon_mae_vals.append(mean_absolute_error(x_np, x_np))

        x_cols = [f"x{i+1}" for i in range(x_np_dr.shape[1])]
        if len(optim_direc) > 1:
            y_cols = [f"y{i+1}" for i in range(y_np.shape[1])]
        else:
            y_cols = ['y1']
            y_np = y_np.reshape(len(y_np), 1)

        df = pd.DataFrame(np.hstack([x_np_dr, y_np]), columns=x_cols + y_cols)

        if isinstance(function, RoelfesEmulator) and function.objective == "coef":
            cand, pred = bayes_opt_roelfes_coef(df, q, q_method)
            pred = pred.reshape(q, len(optim_direc))
        else:
            model = BayesianOptimization().fit(df, y_cols, optim_direc=optim_direc)
            cand, pred = model.candidates(q, q_sampling_method=q_method)

        # Inverse transform and evaluate candidates
        recon_cand = dr_model.inverse_transform(cand) if dr_method else cand
        rc_tensor = torch.tensor(recon_cand)
        recon_cand = torch.max(torch.min(rc_tensor, function.bounds[1].unsqueeze(0)),
                               function.bounds[0].unsqueeze(0))
        if function.discrete_inds is not None:
            recon_cand[:, function.discrete_inds] = torch.round(recon_cand[:, function.discrete_inds])

        if q > 1:
            new_y = function(recon_cand).numpy().reshape(q, len(optim_direc))
        else:
            new_y = function(recon_cand).numpy()

        recon_cand = rc_tensor.numpy()

        # Evaluate results
        if isinstance(function, RoelfesEmulator):
            best_vals.append(function.yield_calc(recon_cand)[:,-1])
        else:
            scalar_y = np.dot(new_y, weights.T) if len(weights) > 1 else new_y.flatten() * weights[0]
            best_vals.append(np.max(scalar_y))

        pred_np = np.array(pred)
        mae_pred_vals.append(mean_absolute_error(pred_np, new_y))

        x_np = np.vstack([x_np, recon_cand])
        y_np = np.vstack([y_np, new_y])

    return {
        "function_name": function_name,
        "dr_name": dr_name,
        "n_components": n_components,
        "q_key": q_key,
        "x": x_np,
        "best_vals_all_folds": best_vals,
        "mae_vals_all_folds": mae_pred_vals,
        "recon_mae_all_folds": recon_mae_vals
    }

In [6]:
# Prepare parallel jobs
parallel_jobs = []

for function_name, function in tqdm(function_list):
    optim_direc = optim_direc_map[function_name]
    weights = np.array([1 if d == "max" else -1 for d in optim_direc])

    if function_name not in results:
        results[function_name] = {}
        data[function_name] = {}

    for cv_iter in range(cv_amount):
        X = generate_random_inputs(function.bounds, starting_samples_n, function.discrete_inds)
        y = function(X)

        x_np = X.numpy().astype(np.float32)
        y_np = y.numpy().astype(np.float32)
        x_np_backup, y_np_backup = x_np.copy(), y_np.copy()

        for dr_method in dim_red_list:
            dr_name = dr_method.__name__ if dr_method else "None"
            for n_components in test_component_choice_list:
                for q in q_list:
                    relevant_sampling_strategies = sampling_strategies if q > 1 else [None]
                    for q_method in relevant_sampling_strategies:
                        job = delayed(run_single_experiment)(
                            function_name=function_name,
                            function=function,
                            optim_direc=optim_direc,
                            weights=weights,
                            cv_iter=cv_iter,
                            x_np_backup=x_np_backup,
                            y_np_backup=y_np_backup,
                            dr_method=dr_method,
                            dr_name=dr_name,
                            n_components=n_components,
                            q=q,
                            q_method=q_method
                        )
                        parallel_jobs.append(job)

# Batch execution parameters
batch_size = 20  # Adjust based on memory/CPU availability

for i in range(0, len(parallel_jobs), batch_size):
    batch = parallel_jobs[i:i + batch_size]
    outputs = Parallel(n_jobs=-1, verbose=10)(batch)

    for out in outputs:
        fn = out["function_name"]
        dr = out["dr_name"]
        comp = out["n_components"]
        qk = out["q_key"]

        if dr not in results[fn]:
            results[fn][dr] = {}
            data[fn][dr] = {}
        if comp not in results[fn][dr]:
            results[fn][dr][comp] = {}
            data[fn][dr][comp] = {}
        if qk not in results[fn][dr][comp]:
            results[fn][dr][comp][qk] = {
                "x": [],
                "best_vals_all_folds": [],
                "mae_vals_all_folds": [],
                "recon_mae_all_folds": []
            }
            data[fn][dr][comp][qk] = {}

        results[fn][dr][comp][qk]["x"].append(out["x"])
        results[fn][dr][comp][qk]["best_vals_all_folds"].append(out["best_vals_all_folds"])
        results[fn][dr][comp][qk]["mae_vals_all_folds"].append(out["mae_vals_all_folds"])
        results[fn][dr][comp][qk]["recon_mae_all_folds"].append(out["recon_mae_all_folds"])

    del outputs
    gc.collect()


100%|██████████| 7/7 [00:04<00:00,  1.49it/s]
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  4 out of 20 | elapsed:   19.7s remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  7 out of 20 | elapsed:   22.7s remaining:   42.2s
[Parallel(n_jobs=-1)]: Done 10 out of 20 | elapsed:   26.1s remaining:   26.1s
[Parallel(n_jobs=-1)]: Done 13 out of 20 | elapsed:   33.6s remaining:   18.0s
[Parallel(n_jobs=-1)]: Done 16 out of 20 | elapsed:  1.5min remaining:   23.1s
[Parallel(n_jobs=-1)]: Done 20 out of 20 | elapsed:  3.0min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  4 out of 20 | elapsed:   20.6s remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  7 out of 20 | elapsed:   26.7s remaining:   49.7s
[Parallel(n_jobs=-1)]: Done 10 out of 20 | elapsed:   36.2s remaining:   36.2s
[Parallel(n_jobs=-1)]: Done 13 out of 20 | elapsed:   40.9s remaining:   22.0s
[Parallel(n_jobs=-1)

In [7]:
# Final aggregation
for fname in results:
    for dr in results[fname]:
        for comp in results[fname][dr]:
            for q_key in results[fname][dr][comp]:
                data[fname][dr][comp][q_key]["best_vals_avg"] = np.mean(
                    results[fname][dr][comp][q_key]["best_vals_all_folds"], axis=0)
                data[fname][dr][comp][q_key]["mae_vals_avg"] = np.mean(
                    results[fname][dr][comp][q_key]["mae_vals_all_folds"], axis=0)
                data[fname][dr][comp][q_key]["recon_mae_avg"] = np.mean(
                    results[fname][dr][comp][q_key]["recon_mae_all_folds"], axis=0)


In [8]:
import pickle

In [9]:
with open('second_attempt_first_experiment.pkl', 'wb') as f:
    pickle.dump(data, f)